In [1]:
import numpy as np
import glob, os
import matplotlib.pyplot as plt
import pandas as pd


def read_all_runs(dir_path):
    
    files = sorted(glob.glob(os.path.join(dir_path, "*.npz")))
    all_rs, all_ps, all_bs = [], [], []

    print(f"Found {len(files)} files in {dir_path}")

    for f in files:
        data = np.load(f)
        
        if 'rewards' in data:
            res_r = data['rewards']
            res_p = data['powers']
            res_b = data['buffers']
        else:
            res_r = data["arr_0"]
            res_p = data["arr_1"]
            res_b = data["arr_2"]

        all_rs.append(res_r)
        all_ps.append(res_p)
        all_bs.append(res_b)

    return np.array(all_rs), np.array(all_ps), np.array(all_bs)



def moving_average(data, window):
    return np.convolve(data, np.ones(window), 'valid') / window


def plot_multi_algorithm_comparison(algo_data_dict, FIG_DIR, win=10, metric='reward'):
    
    
    # Lấy số lượng users từ algorithm đầu tiên
    first_algo_data = list(algo_data_dict.values())[0]
    num_users = first_algo_data.shape[2]
    num_episodes = first_algo_data.shape[1]
    
    # Colors và line styles cho các thuật toán
    algo_names = list(algo_data_dict.keys())
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']  # blue, orange, green, red
    linestyles = ['-', '-', '--', '--']
    
    # Metric labels
    metric_labels = {
        'reward': 'Average Reward',
        'power': 'Average Power (W)',
        'delay': 'Average Delay (ms)'
    }
    ylabel = metric_labels.get(metric, 'Average Reward')
    
   
    os.makedirs(FIG_DIR, exist_ok=True)
    
    
    for user_idx in range(num_users):
        # ===== RAW CURVES =====
        plt.figure(figsize=(10, 6))
        
        for idx, (algo_name, data) in enumerate(algo_data_dict.items()):
            # Mean over runs: (num_episodes, num_users)
            data_mean = np.mean(data, axis=0)
            # Lấy data của user này
            user_data = data_mean[:, user_idx]
            
            plt.plot(
                user_data,
                alpha=0.6,
                linewidth=1.5,
                color=colors[idx],
                linestyle=linestyles[idx],
                label=algo_name
            )
        
        plt.xlabel("Episode", fontsize=12)
        plt.ylabel(ylabel, fontsize=12)
        plt.title(f"User {user_idx+1} - Algorithm Comparison (Raw)", fontsize=14)
        plt.legend(loc='best', fontsize=11)
        plt.grid(True, alpha=0.3)
        
        plt.xticks(np.arange(0, num_episodes + 1, 250))
        ax = plt.gca()
        for spine in ax.spines.values():
            spine.set_edgecolor('black')
            spine.set_linewidth(1.2)
        
        plt.tight_layout()
        
        fig_path = os.path.join(FIG_DIR, f"user_{user_idx+1}_{metric}_raw.png")
        plt.savefig(fig_path, dpi=300, bbox_inches="tight")
        plt.close()
        print(f"✓ Saved: {fig_path}")
        
        # ===== SMOOTHED CURVES =====
        plt.figure(figsize=(10, 6))
        
        for idx, (algo_name, data) in enumerate(algo_data_dict.items()):
            data_mean = np.mean(data, axis=0)
            user_data = data_mean[:, user_idx]
            
            # Smooth
            sm_curve = moving_average(user_data, win)
            
            plt.plot(
                range(win - 1, num_episodes),
                sm_curve,
                linewidth=2.5,
                color=colors[idx],
                linestyle=linestyles[idx],
                label=algo_name
            )
        
        plt.xlabel("Episode", fontsize=12)
        plt.ylabel(ylabel, fontsize=12)
        plt.title(f"User {user_idx+1} - Algorithm Comparison (Smoothed, win={win})", fontsize=14)
        plt.legend(loc='best', fontsize=11)
        plt.grid(True, alpha=0.3)
        
        plt.xticks(np.arange(0, num_episodes + 1, 250))
        ax = plt.gca()
        for spine in ax.spines.values():
            spine.set_edgecolor('black')
            spine.set_linewidth(1.2)
        
        plt.tight_layout()
        
        fig_path = os.path.join(FIG_DIR, f"user_{user_idx+1}_{metric}_smoothed.png")
        plt.savefig(fig_path, dpi=300, bbox_inches="tight")
        plt.close()
        print(f"✓ Saved: {fig_path}")
        
        # ===== COMBINED (RAW + SMOOTHED) =====
        plt.figure(figsize=(12, 6))
        
        for idx, (algo_name, data) in enumerate(algo_data_dict.items()):
            data_mean = np.mean(data, axis=0)
            user_data = data_mean[:, user_idx]
            
            # Raw (faded)
            plt.plot(
                user_data,
                alpha=0.25,
                linewidth=1,
                color=colors[idx],
                linestyle=linestyles[idx]
            )
            
            # Smoothed (bold)
            sm_curve = moving_average(user_data, win)
            plt.plot(
                range(win - 1, num_episodes),
                sm_curve,
                linewidth=2.5,
                color=colors[idx],
                linestyle=linestyles[idx],
                label=algo_name
            )
        
        plt.xlabel("Episode", fontsize=12)
        plt.ylabel(ylabel, fontsize=12)
        plt.title(f"User {user_idx+1} - Algorithm Comparison (win={win})", fontsize=14)
        plt.legend(loc='best', fontsize=11)
        plt.grid(True, alpha=0.3)
        
        plt.xticks(np.arange(0, num_episodes + 1, 250))
        ax = plt.gca()
        for spine in ax.spines.values():
            spine.set_edgecolor('black')
            spine.set_linewidth(1.2)
        
        plt.tight_layout()
        
        fig_path = os.path.join(FIG_DIR, f"user_{user_idx+1}_{metric}_combined.png")
        plt.savefig(fig_path, dpi=300, bbox_inches="tight")
        plt.close()
        print(f"✓ Saved: {fig_path}")



def plot_each_user_separate_figure(algo_data_dict, FIG_DIR, win=10, metric='reward', 
                                    arrival_rates=None):
    
    
    first_algo_data = list(algo_data_dict.values())[0]
    num_users = first_algo_data.shape[2]
    num_episodes = first_algo_data.shape[1]
    
    algo_names = list(algo_data_dict.keys())
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']
    linestyles = ['-', '-', '--', '--']
    
    metric_labels = {
        'reward': 'Average Reward',
        'power': 'Average Power (W)',
        'delay': 'Average Delay (ms)'
    }
    ylabel = metric_labels.get(metric, 'Average Reward')
    
    os.makedirs(FIG_DIR, exist_ok=True)
    
    # ===== VẼ TỪNG USER RIÊNG BIỆT =====
    for user_idx in range(num_users):
        plt.figure(figsize=(10, 6))
        
        for idx, (algo_name, data) in enumerate(algo_data_dict.items()):
            data_mean = np.mean(data, axis=0)
            user_data = data_mean[:, user_idx]
            sm_curve = moving_average(user_data, win)
            
            plt.plot(
                range(win - 1, num_episodes),
                sm_curve,
                linewidth=2.5,
                color=colors[idx],
                linestyle=linestyles[idx],
                label=algo_name
            )
        
        # Title với arrival rate nếu có
        if arrival_rates is not None:
            plt.title(f"User {user_idx+1} (λ={arrival_rates[user_idx]:.1f} Mbps)", 
                     fontsize=14, fontweight='bold')
        else:
            plt.title(f"User {user_idx+1}", fontsize=14, fontweight='bold')
        
        plt.xlabel("Episode", fontsize=12)
        plt.ylabel(ylabel, fontsize=12)
        plt.legend(loc='best', fontsize=11)
        plt.grid(True, alpha=0.3)
        
        plt.xticks(np.arange(0, num_episodes + 1, 250))
        ax = plt.gca()
        for spine in ax.spines.values():
            spine.set_edgecolor('black')
            spine.set_linewidth(1.2)
        
        plt.tight_layout()
        
        fig_path = os.path.join(FIG_DIR, f"user_{user_idx+1}_{metric}_algo_comparison.png")
        plt.savefig(fig_path, dpi=300, bbox_inches="tight")
        plt.close()
        print(f"✓ Saved: {fig_path}")









def main():
    RESULT_DIRS = {
        'DDPG': 'ddpg/results_ddpg_pure_training',   
        'DDPG+IDE': 'ddpg_ide/results_ddpg_ide_deterministic',
        
    }
    
    
    FIG_DIR = 'training_figures/'
    
    
    WINDOW = 10
    
    
    ARRIVAL_RATES = [1.0, 2.0, 3.0]  
    
    
    print("Reading data from all algorithms...")
    
    algo_rewards = {}
    algo_powers = {}
    algo_delays = {}
    
    for algo_name, dir_path in RESULT_DIRS.items():
        print(f"\n--- Loading {algo_name} ---")
        all_rs, all_ps, all_bs = read_all_runs(dir_path)
        
        algo_rewards[algo_name] = all_rs
        algo_powers[algo_name] = all_ps
        algo_delays[algo_name] = all_bs
        
        print(f"  Shape: {all_rs.shape}")
    
    
    print(f"\n{'='*60}")
    print("PLOTTING COMPARISONS...")
    print(f"{'='*60}\n")
    
    

    
   
    print("\n4. Plotting each user in separate figures (reward)...")
    plot_each_user_separate_figure(
        algo_rewards,
        FIG_DIR,
        win=WINDOW,
        metric='reward',
        arrival_rates=ARRIVAL_RATES
    )
    
    
    print("\n5. Plotting each user in separate figures (power)...")
    plot_each_user_separate_figure(
        algo_powers,
        FIG_DIR,
        win=WINDOW,
        metric='power',
        arrival_rates=ARRIVAL_RATES
    )
    
    
    print("\n6. Plotting each user in separate figures (delay)...")
    plot_each_user_separate_figure(
        algo_delays,
        FIG_DIR,
        win=WINDOW,
        metric='delay',
        arrival_rates=ARRIVAL_RATES
    )
    
    
   




In [2]:
main()

Reading data from all algorithms...

--- Loading DDPG ---
Found 5 files in ddpg/results_ddpg_pure_training
  Shape: (5, 1500, 3)

--- Loading DDPG+IDE ---
Found 7 files in ddpg_ide/results_ddpg_ide_deterministic
  Shape: (7, 1500, 3)

PLOTTING COMPARISONS...


4. Plotting each user in separate figures (reward)...
✓ Saved: training_figures/user_1_reward_algo_comparison.png
✓ Saved: training_figures/user_2_reward_algo_comparison.png
✓ Saved: training_figures/user_3_reward_algo_comparison.png

5. Plotting each user in separate figures (power)...
✓ Saved: training_figures/user_1_power_algo_comparison.png
✓ Saved: training_figures/user_2_power_algo_comparison.png
✓ Saved: training_figures/user_3_power_algo_comparison.png

6. Plotting each user in separate figures (delay)...
✓ Saved: training_figures/user_1_delay_algo_comparison.png
✓ Saved: training_figures/user_2_delay_algo_comparison.png
✓ Saved: training_figures/user_3_delay_algo_comparison.png


In [9]:

def summarize_per_user(all_rs, all_ps, all_bs):
    """
    Input shape:
        (num_runs, num_episodes, num_users)
    Output:
        Pandas DataFrame (per-user + overall)
    """

    num_runs, num_episodes, num_users = all_rs.shape

    # Mean over runs & episodes
    avg_r = np.mean(all_rs, axis=(0, 1))
    avg_p = np.mean(all_ps, axis=(0, 1))
    avg_b = np.mean(all_bs, axis=(0, 1))

    df = pd.DataFrame({
        "User": [f"User {i+1}" for i in range(num_users)],
        "Reward (avg)": avg_r,
        "Power (avg, W)": avg_p,
        "Delay (avg, ms)": avg_b
    })

    # Overall average row
    df.loc[len(df)] = [
        "Average (all users)",
        np.mean(avg_r),
        np.mean(avg_p),
        np.mean(avg_b)
    ]

    return df




In [11]:
dir_path = "ddpg/test_results_ddpg_pure_training"
all_rs, all_ps, all_bs = read_all_runs(dir_path)
df_ddpg = summarize_per_user(all_rs, all_ps, all_bs)


print("\n=== DDPG Average Results (All Runs & Episodes) ===\n")
print(
    df_ddpg.to_string(
        index=False,
        formatters={
            "Reward (avg)": "{:.3f}".format,
            "Power (avg, W)": "{:.3f}".format,
            "Delay (avg, ms)": "{:.3f}".format
        }
    )
)


Found 5 files in ddpg/test_results_ddpg_pure_training

=== DDPG Average Results (All Runs & Episodes) ===

               User Reward (avg) Power (avg, W) Delay (avg, ms)
             User 1       -2.130          0.202           2.189
             User 2      -10.522          1.354           4.846
             User 3      -16.589          2.430           8.607
Average (all users)       -9.747          1.329           5.214


In [10]:
dir_path = "ddpg_ide/test_results_ddpg_ide_deterministic"
all_rs, all_ps, all_bs = read_all_runs(dir_path)
df_ddpg = summarize_per_user(all_rs, all_ps, all_bs)


print("\n=== DDPG+IDE Average Results (All Runs & Episodes) ===\n")
print(
    df_ddpg.to_string(
        index=False,
        formatters={
            "Reward (avg)": "{:.3f}".format,
            "Power (avg, W)": "{:.3f}".format,
            "Delay (avg, ms)": "{:.3f}".format
        }
    )
)


Found 7 files in ddpg_ide/test_results_ddpg_ide_deterministic

=== DDPG+IDE Average Results (All Runs & Episodes) ===

               User Reward (avg) Power (avg, W) Delay (avg, ms)
             User 1       -1.885          0.200           1.633
             User 2       -6.124          0.784           3.999
             User 3      -13.903          1.789           8.711
Average (all users)       -7.304          0.924           4.781
